<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/utilities/examples_MLFlow_InColab_with_NGROK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import our helper Library

In [ ]:
%load_ext autoreload
%autoreload 2

!pip install mlflow --quiet
!pip install pyngrok --quiet

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# add the path to the python library path
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.6/505.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0

Start our MLFlow server in the background and expose the endpoint using nGROk

In [ ]:
import ColabLibrary

url_mlflow=ColabLibrary.startMLFlowUI()
print (url_mlflow)

MLflow Tracking UI: https://7680-34-145-94-238.ngrok-free.app
https://7680-34-145-94-238.ngrok-free.app


In [ ]:
import mlflow

mlflow.set_tracking_uri(uri=url_mlflow)

# Create a new MLflow Experiment
mlflow.set_experiment("ColabLibrary example")

2024/07/31 15:23:22 INFO mlflow.tracking.fluent: Experiment with name 'ColabLibrary example' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/686960622834172286', creation_time=1722439402700, experiment_id='686960622834172286', last_update_time=1722439402700, lifecycle_stage='active', name='ColabLibrary example', tags={}>

Now we can just implement a quick small test.Lets train the wellknown IRIS model:


In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
   #  "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
model = LogisticRegression(**params)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)



Now we can upload the metrics and the model to mlflow

In [ ]:
from mlflow.models import infer_signature

mlflow.set_experiment("IRIS Logistic Regression")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="Iris-Dataset-Logistic-Regression-test",
    )

2024/07/31 15:30:44 INFO mlflow.tracking.fluent: Experiment with name 'IRIS Logistic Regression' does not exist. Creating a new experiment.
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'Iris-Dataset-Logistic-Regression-test'.
2024/07/31 15:30:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Iris-Dataset-Logistic-Regression-test, version 1
Created version '1' of model 'Iris-Dataset-Logistic-Regression-test'.


2024/07/31 15:30:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run worried-mule-364 at: https://7680-34-145-94-238.ngrok-free.app/#/experiments/588577370895502167/runs/9cb7c76e5e364584b4182f9f3182ed05.
2024/07/31 15:30:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://7680-34-145-94-238.ngrok-free.app/#/experiments/588577370895502167.
